In [ ]:
import numpy as np

# --- Conversion functions as provided ---

def augment(v):
    # v is (N, 3); this returns an (N, 4) array with ones in the last column.
    return np.concatenate([v, np.ones_like(v[:, :1])], axis=-1)

def augmentT(v):
    # v is (M, N); this returns an (M+1, N) array with an extra row of ones.
    return np.concatenate([v, np.ones_like(v[:1])], axis=0)

def point2image(vertices, viewmat, projection_matrix, cam_pos, eps=np.finfo(np.float32).eps):
    """
    Convert a set of 3D vertices to 2D pixel-space coordinates plus the per-vertex inverse distance.
    
    vertices: (N,3)
    viewmat: (4,4)
    projection_matrix: (3,3)
    cam_pos: (3,)
    
    Returns an (N, 3) array where each row is [x, y, inv_distance].
    """
    # Transform vertices into camera space (homogeneous coordinates)
    cam_space_homo = viewmat @ augment(vertices).T  # shape (4, N)
    # Perform perspective division on the first two coordinates using the z coordinate.
    cam_space_nohomo = cam_space_homo[:2] / (np.abs(cam_space_homo[2:3]) + 1e-10)
    # Map to pixel space (here using an identity projection_matrix)
    pixel_space = projection_matrix @ augmentT(cam_space_nohomo)  # shape (3, N)
    # Compute the inverse of the distance from each vertex to the camera.
    inv_distance = 1 / np.clip(cam_space_homo[2:3].T, eps, None)
    # inv_distance = 1 / np.sqrt(
    #     np.clip(np.sum((vertices - cam_pos.reshape(1, 3))**2, axis=1, keepdims=True), eps, None)
    # )
    # Concatenate the 2D positions with the inverse distance.
    return np.concatenate([pixel_space[:2].T, inv_distance], axis=1)

# --- 2D Ray–Triangle Intersection (using barycentrics) ---

def ray_triangle_intersect_2D(pixelPos, t0, t1, t2, w0, w1, w2):
    """
    Given a 2D pixel position and a 2D triangle (with w=1/distance at each vertex),
    compute the perspective-correct distance along the ray.
    
    Returns (hit, distance) where hit is True if the pixel is inside the triangle.
    """
    # Ensure inputs are numpy arrays.
    pixelPos = np.asarray(pixelPos, dtype=np.float32)
    t0 = np.asarray(t0, dtype=np.float32)
    t1 = np.asarray(t1, dtype=np.float32)
    t2 = np.asarray(t2, dtype=np.float32)
    
    # Compute edge vectors from t0.
    e0 = t1 - t0
    e1 = t2 - t0
    v  = pixelPos - t0

    # Compute dot products.
    dot00 = np.dot(e0, e0)
    dot01 = np.dot(e0, e1)
    dot02 = np.dot(e0, v)
    dot11 = np.dot(e1, e1)
    dot12 = np.dot(e1, v)

    # Compute barycentrics.
    denom = dot00 * dot11 - dot01 * dot01
    alpha = (dot11 * dot02 - dot01 * dot12) / denom
    beta  = (dot00 * dot12 - dot01 * dot02) / denom
    gamma = 1.0 - alpha - beta

    # If any barycentric coordinate is negative, the pixel is outside the triangle.
    if alpha < 0.0 or beta < 0.0 or gamma < 0.0:
        return False, None

    # Interpolate the reciprocal distance.
    reciprocalDist = alpha * w1 + beta * w2 + gamma * w0
    if reciprocalDist <= 0.0:
        return False, None

    # Final perspective-correct distance.
    distance = 1.0 / reciprocalDist
    return True, distance

# --- 3D Ray–Triangle Intersection (Möller–Trumbore algorithm) ---

def safe_div(a, b, eps=1e-8):
    return a / b if abs(b) >= eps else 0.0

def ray_triangle_intersect3D(ray_origin, ray_vector, tri_a, tri_b, tri_c):
    """
    Compute the 3D ray-triangle intersection using the Möller–Trumbore algorithm.
    
    Returns (hit, t) where t is the ray parameter at the intersection.
    """
    ray_origin = np.asarray(ray_origin, dtype=np.float32)
    ray_vector = np.asarray(ray_vector, dtype=np.float32)
    tri_a = np.asarray(tri_a, dtype=np.float32)
    tri_b = np.asarray(tri_b, dtype=np.float32)
    tri_c = np.asarray(tri_c, dtype=np.float32)

    edge1 = tri_b - tri_a
    edge2 = tri_c - tri_a
    ray_cross_e2 = np.cross(ray_vector, edge2)
    det = np.dot(edge1, ray_cross_e2)
    inv_det = safe_div(1.0, det)
    s = ray_origin - tri_a
    u = inv_det * np.dot(s, ray_cross_e2)
    if u < 0 or u > 1:
        return False, None
    s_cross_e1 = np.cross(s, edge1)
    v = inv_det * np.dot(ray_vector, s_cross_e1)
    if v < 0 or (u + v) > 1:
        return False, None
    t = inv_det * np.dot(edge2, s_cross_e1)
    return True, t

# --- Testing the 2D and 3D intersections ---

if __name__ == "__main__":
    # Camera and projection setup.
    cam_pos = np.array([0.0, 0.0, 0.0], dtype=np.float32)
    viewmat = np.eye(4, dtype=np.float32)  # Identity view matrix.
    projection_matrix = np.eye(3, dtype=np.float32)  # Identity projection (i.e. no additional mapping).

    # Define a 3D triangle (in front of the camera at z = 5).
    vertices = np.array([
        [1.0,  1.0,  5.0],
        [-1.0, 1.0,  5.0],
        [0.0,  -1.0, 5.0]
    ], dtype=np.float32)

    # Convert the 3D triangle to a 2D representation.
    pts_2d = point2image(vertices, viewmat, projection_matrix, cam_pos)
    # pts_2d is (3, 3): each row is [x, y, inv_distance]
    
    print("2D triangle vertices (pixel coordinates and inv_distance):")
    print(pts_2d)
    
    # Choose a test pixel: for example, the barycenter of the 2D triangle.
    pixel_pos = np.mean(pts_2d[:, :2], axis=0)
    print("\nTest pixel (barycenter in 2D):", pixel_pos)

    # --- 2D Intersection ---
    hit2D, depth = ray_triangle_intersect_2D(
        pixel_pos,
        pts_2d[0, :2],
        pts_2d[1, :2],
        pts_2d[2, :2],
        pts_2d[0, 2],
        pts_2d[1, 2],
        pts_2d[2, 2]
    )
    print("\n2D Intersection:")
    print("  Hit:", hit2D, "Distance:", distance2D)

    # --- 3D Intersection ---
    # In our simplified setup, the conversion from 3D to 2D is:
    #   pixel = [x/z, y/z]
    # so we can reconstruct a ray direction as [pixel_x, pixel_y, 1] (then normalize).
    ray_dir = np.array([pixel_pos[0], pixel_pos[1], 1.0], dtype=np.float32)
    ray_dir /= np.linalg.norm(ray_dir)
    print("\nReconstructed ray direction from pixel:", ray_dir)
    
    hit3D, distance3D = ray_triangle_intersect3D(cam_pos, ray_dir, vertices[0], vertices[1], vertices[2])
    print("\n3D Intersection:")
    print("  Hit:", hit3D, "Distance:", distance3D)

    # --- Compare the distances ---
    if hit2D and hit3D:
        diff = abs(distance2D - distance3D)
        print("\nDifference between 2D and 3D intersection distances:", diff)
    else:
        print("\nOne or both intersections failed.")


2D triangle vertices (pixel coordinates and inv_distance):
[[ 0.2  0.2  0.2]
 [-0.2  0.2  0.2]
 [ 0.  -0.2  0.2]]

Test pixel (barycenter in 2D): [0.         0.06666667]

2D Intersection:
  Hit: True Distance: 5.000000124176346

Reconstructed ray direction from pixel: [0.         0.06651901 0.9977851 ]

3D Intersection:
  Hit: True Distance: 5.011099126550115

Difference between 2D and 3D intersection distances: 0.011099002373769373
